In [113]:
targets = {
    'AMZN':0.40,
    'CSCO':0.30,
    'GE':0.30
}

In [114]:
import pandas as pd
import numpy as np

portfolio = pd.DataFrame(
    index=list(targets.keys()) + ['CASH'],
    data={
        'data':'2018-01-01',
        'price': [np.NaN, np.NaN, np.NaN, 1],
        'target': [0.4, 0.3, 0.3, 0],
        'allocation': [0, 0, 0, 1],
        'shares': [0, 0, 0, 10000],
        'market_value': [0, 0, 0, 10000]
    
    }
)

print(portfolio)

            data  price  target  allocation  shares  market_value
AMZN  2018-01-01    NaN     0.4           0       0             0
CSCO  2018-01-01    NaN     0.3           0       0             0
GE    2018-01-01    NaN     0.3           0       0             0
CASH  2018-01-01    1.0     0.0           1   10000         10000


In [115]:
def instantiate_portfolio(targtes, starting_balance):
    targets['CASH'] = 0
    tickers = list(targets.keys())
    
    df = pd.DataFrame(
        index=tickers,
        columns=[
            'data', 'price', 'target', 'allocation', 'shares', 'market_value'
        ]
    )
    df.shares = 0
    df.market_value = 0
    df.allocation = 0
    df.update(
        pd.DataFrame
            .from_dict(targets, orient='index')
            .rename(columns={0:'target'})
    )

    df.at['CASH', 'shares'] = starting_balance

    return df

In [116]:
portfolio = instantiate_portfolio(
    {'AMZN':0.4, 'CSCO':0.3, 'GE':0.3},
    10000
)
print(portfolio)

     data price target  allocation  shares  market_value
AMZN  NaN   NaN    0.4           0       0             0
CSCO  NaN   NaN    0.3           0       0             0
GE    NaN   NaN    0.3           0       0             0
CASH  NaN   NaN      0           0   10000             0


In [117]:
print(targets)
targets['CASH'] = 0
print(targets)

{'AMZN':0.4, 'CSCO':0.3, 'GE':0.3}
{'AMZN':0.4, 'CSCO':0.3, 'GE':0.3, 'CASH':0}

{'AMZN': 0.4, 'CSCO': 0.3, 'GE': 0.3, 'CASH': 0}
{'AMZN': 0.4, 'CSCO': 0.3, 'GE': 0.3, 'CASH': 0}


{'AMZN': 0.4, 'CSCO': 0.3, 'GE': 0.3, 'CASH': 0}

In [133]:
list(targets.keys())

['AMZN', 'CSCO', 'GE']

['AMZN', 'CSCO', 'GE']

In [119]:
print(pd.DataFrame.from_dict(targets, orient='index'))

        0
AMZN  0.4
CSCO  0.3
GE    0.3
CASH  0.0


In [120]:
print(
pd.DataFrame
    .from_dict(targets, orient='index')
    .rename(columns={0:'target'})
)

      target
AMZN     0.4
CSCO     0.3
GE       0.3
CASH     0.0


In [121]:
df = pd.DataFrame(index=['CASH', 'GE'], data={'shares': [0, 1]})
print(df)
df.at['CASH', 'shares'] = 10000
print(df)

      shares
CASH       0
GE         1
      shares
CASH   10000
GE         1


In [122]:
print(portfolio)

     data price target  allocation  shares  market_value
AMZN  NaN   NaN    0.4           0       0             0
CSCO  NaN   NaN    0.3           0       0             0
GE    NaN   NaN    0.3           0       0             0
CASH  NaN   NaN      0           0   10000             0


In [123]:
def update_prices(portfolio, prices):
    prices['CASH'] = 1
    portfolio.update(pd.DataFrame({'price': prices}))
    portfolio.date = prices.name
    portfolio.market_value = portfolio.shares * portfolio.price

In [124]:
prices = pd.Series(
    name='2018-01-01', 
    data={'AMZN': 945.21,'CSCO': 30.52, 'GE': 29.27}
)
print(prices)
update_prices(portfolio, prices)

AMZN    945.21
CSCO     30.52
GE       29.27
Name: 2018-01-01, dtype: float64


In [125]:
print(portfolio)

     data   price target  allocation  shares market_value
AMZN  NaN  945.21    0.4           0       0            0
CSCO  NaN   30.52    0.3           0       0            0
GE    NaN   29.27    0.3           0       0            0
CASH  NaN       1      0           0   10000        10000


In [126]:
def get_order(portfolio):
    total_value = portfolio.market_value.sum()
    
    order = (
        (total_value * portfolio.target // portfolio.price)
        - portfolio.shares
    ).drop('CASH')
    return order

In [127]:
total_value = 10000
target = 0.4
price = 945.21
AMZN = (total_value * target // price) - 0
print(AMZN)

4.0


In [128]:
order = get_order(portfolio)
print(order)

AMZN      4
CSCO     98
GE      102
dtype: object


In [129]:
def deposit(portfolio, amount):
    portfolio.at['CASH', 'shares'] += amount
    portfolio.at['CASH', 'market_value'] = portfolio.at['CASH', 'shares']
deposit(portfolio, 1000)

In [130]:
order = get_order(portfolio)
print(order)

AMZN      4
CSCO    108
GE      112
dtype: object


In [131]:
def simulate_process_order(portfolio, order):
    starting_cash = portfolio.at['CASH', 'shares']
    cash_adjustment = np.sum(order * portfolio.price)
    portfolio.shares += order
    portfolio.market_value = portfolio.shares * portfolio.price
    portfolio.at['CASH', 'shares'] = starting_cash - cash_adjustment
    portfolio.market_value = portfolio.shares * portfolio.price
    portfolio.allocation = (
        portfolio.market_value / portfolio.market_value.sum()
    )

In [132]:
simulate_process_order(portfolio, order)
print(portfolio)

     data   price target allocation  shares market_value
AMZN  NaN  945.21    0.4   0.343713       4      3780.84
CSCO  NaN   30.52    0.3   0.299651     108      3296.16
GE    NaN   29.27    0.3   0.298022     112      3278.24
CASH  NaN       1      0  0.0586145  644.76       644.76
